In [4]:
SparkContext

pyspark.context.SparkContext

In [5]:
!pwd

/home/almalinux/eda1-coursework/src/merizo_pipeline


In [6]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://download.pytorch.org/whl/cpu/torch-2.0.1%2Bcpu-cp39-cp39-linux_x86_64.whl (195.4 MB)


In [7]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [8]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [9]:
sc.master

'local[3]'

In [10]:
os.getcwd()

'/home/almalinux/eda1-coursework/src/merizo_pipeline'

In [11]:
def read_dir(input_dir, output_dir):
    """
    Read file paths from HDFS using SparkContext.
    """
    # Use SparkContext to read files from HDFS
    # If files are located in a directory on HDFS, you can use textFile 
    file_rdd = sc.wholeTextFile(input_dir + "/AF-Q46839-F1-model_v4.pdb") 
    file_paths = file_rdd.collect()  # This retrieves the file paths as a list

    # Create a list of tuples with file path, id, and output directory
    return [(file_path, os.path.basename(file_path), output_dir) for file_path in file_paths]


In [12]:
input_dir = "/UP000000625_83333_ECOLI_v4/"

In [13]:
file_rdd = sc.wholeTextFiles(input_dir + "*.pdb")
file_paths_rdd = file_rdd.map(lambda x: (x[0], os.path.basename(x[0])))

In [15]:
local_file_path = SparkFiles.get(input_dir + "/AF-Q46839-F1-model_v4.pdb")
local_file_path

NameError: name 'SparkFiles' is not defined

In [16]:
!ls

IOStream.flush timed out
merizo_pipeline  pipeline_playground.ipynb  results_parser.py
pipeline_job.py  requirements.txt	    setup.py


In [17]:
!pwd

IOStream.flush timed out
/home/almalinux/eda1-coursework/src/merizo_pipeline


In [167]:
run_merizo_search(('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb', 'AF-Q46839-F1-model_v4.pdb'))

File Name: AF-Q46839-F1-model_v4.pdb, File Path: hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb
RUNNINGGGG MERIZOO SEARCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
STEP 1: RUNNING MERIZO: python3 /home/almalinux/merizo_search/merizo_search/merizo.py easy-search hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb /home/almalinux/data/cath-4.3-foldclassdb AF-Q46839-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1


In [170]:
!ls

IOStream.flush timed out
__init__.py  pipeline_playground.ipynb	pipeline_script.py
parser.py    pipeline.py		results_parser.py


In [169]:
!rm -rf AF-Q46839-F1-model_v4.pdb AF-Q46839-F1-model_v4.pdb_segment.tsv 

IOStream.flush timed out


In [18]:
input_dir = "/UP000000625_83333_ECOLI_v4/"
file_rdd = sc.binaryFiles(input_dir + "*.pdb")
file_rdd = file_rdd.sample(withReplacement=False, fraction=0.005)
file_content_rdd = file_rdd.map(lambda x: (os.path.basename(x[0]), x[1]))

In [19]:
from subprocess import Popen,run, PIPE
from tempfile import NamedTemporaryFile
from pyspark.sql import SparkSession
import os

def run_merizo_search(file_tuple):
    input_file_name, file_content = file_tuple
    print(f"File Name: {input_file_name}")

    # Create a temporary file to hold the content
    with NamedTemporaryFile(delete=True, mode='wb') as temp_file:
        temp_file.write(file_content)
        temp_file_path = temp_file.name
        cmd = ['python3',
           '/home/almalinux/merizo_search/merizo_search/merizo.py',
           'easy-search',
           temp_file_path,
           '/home/almalinux/data/cath-4.3-foldclassdb',
           input_file_name,
           'tmp',
           '--iterate',
           '--output_headers',
           '-d',
           'cpu',
           '--threads',
           '1'
          ]
        print(f'STEP 1: RUNNING MERIZO: {" ".join(cmd)}')
        p = Popen(cmd, stdin=PIPE,stdout=PIPE, stderr=PIPE)
        out, err = p.communicate()
        # Decode the byte output to string
        print("Output:")
        print(out.decode())  # Decode and print the standard output
        
        if err:
            print("Error:")
            print(err.decode())  # Decode and print the standard error
            

In [ ]:
file_content_rdd.map(run_merizo_search).collect()

In [24]:
def run_parser(input_file):
    """
    Run the results_parser.py over the hhr file to produce the output summary
    """
    search_file = input_file+"_search.tsv"
    print("search_file: ", search_file)
    cmd = ['python3', './results_parser.py', search_file]
    print(f'STEP 2: RUNNING PARSER: {" ".join(cmd)}')
    p = Popen(cmd, stdin=PIPE,stdout=PIPE, stderr=PIPE)
    out, err = p.communicate()
    # Decode the byte output to string
    print("Output:")
    print(out.decode("utf-8"))  # Decode and print the standard output
        
    if err:
        print("Error:")
        print(err.decode("utf-8"))  # Decode and print the standard error

Output:

Error:
2024-12-07 03:38:11,452 | INFO | Starting easy-search with command: 

/home/almalinux/merizo_search/merizo_search/merizo.py easy-search /tmp/tmpj6817fqh /home/almalinux/data/cath-4.3-foldclassdb AF-P0A8A4-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1

2024-12-07 03:38:20,676 | INFO | Finished easy-search in 9.223 seconds.



In [25]:
def run_merizo_search(file_name, file_content):
    print(f"File Name: {file_name}")
    # Create a temporary file to hold the content
    with NamedTemporaryFile(delete=True, mode='wb') as temp_file:
        temp_file.write(file_content)
        temp_file_path = temp_file.name
        cmd = ['python3',
           '/home/almalinux/merizo_search/merizo_search/merizo.py',
           'easy-search',
           temp_file_path,
           '/home/almalinux/data/cath-4.3-foldclassdb',
           file_name,
           'tmp',
           '--iterate',
           '--output_headers',
           '-d',
           'cpu',
           '--threads',
           '1'
          ]
        print(f'STEP 1: RUNNING MERIZO: {" ".join(cmd)}')
        p = Popen(cmd, stdin=PIPE,stdout=PIPE, stderr=PIPE)
        out, err = p.communicate()
        # Decode the byte output to string
        print("Output:")
        print(out.decode("utf-8"))  # Decode and print the standard output
        
        if err:
            print("Error:")
            print(err.decode("utf-8"))  # Decode and print the standard error


[Stage 0:==========================================>                (5 + 2) / 7]

In [26]:

def pipeline(file_tuple):
    file_name, file_content = file_tuple
    # STEP 1
    run_merizo_search(file_name, file_content)
    # STEP 2
    run_parser(file_name)

Output:

Error:
2024-12-07 03:38:21,873 | INFO | Starting easy-search with command: 

/home/almalinux/merizo_search/merizo_search/merizo.py easy-search /tmp/tmptojva8uj /home/almalinux/data/cath-4.3-foldclassdb AF-P76334-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1

2024-12-07 03:38:27,786 | INFO | Finished easy-search in 5.913 seconds.



In [27]:
file_content_rdd.map(pipeline).collect()


Output:

Error:
2024-12-07 03:38:16,696 | INFO | Starting easy-search with command: 

/home/almalinux/merizo_search/merizo_search/merizo.py easy-search /tmp/tmpumku45eo /home/almalinux/data/cath-4.3-foldclassdb AF-P27830-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1

2024-12-07 03:38:28,425 | INFO | Finished easy-search in 11.729 seconds.

File Name: AF-P31224-F1-model_v4.pdb
STEP 1: RUNNING MERIZO: python3 /home/almalinux/merizo_search/merizo_search/merizo.py easy-search /tmp/tmpv3iqi9p8 /home/almalinux/data/cath-4.3-foldclassdb AF-P31224-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1
File Name: AF-P0AAQ6-F1-model_v4.pdb/ 7][Stage 1:>                  (0 + 2) / 7]
STEP 1: RUNNING MERIZO: python3 /home/almalinux/merizo_search/merizo_search/merizo.py easy-search /tmp/tmpdo3werk9 /home/almalinux/data/cath-4.3-foldclassdb AF-P0AAQ6-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1
File Name: AF-P37671-F1-model_v4.pdb
STEP 1: RUNNING M

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]